In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 18 01:45:07 2022

@author: nina
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import make_scorer
from tqdm import tqdm  # For progress tracking
from time import time
import logging
import sys
import itertools

In [14]:
data = pd.read_csv("/content/drive/MyDrive/MLFP/Data/missing_treated_v2.csv")
train_data = data.iloc[:5634, :]
test_data = data.iloc[5634: , :]

### **Drop Unlabelled Data**
Since in the total of 5,634 rows there are around 16% of the data that is unlabeled. It is more suitable to train a **supervised model** with the labeled data, and maybe moved to semi-supervised model in the future. Therefore, I dropped the unlabelled data.

In [15]:
unlabeled_train_data = train_data[train_data.isna().any(axis=1)]
unlabeled_index = unlabeled_train_data.index
new_train_data = train_data.drop(unlabeled_index, axis=0)

### **Train-test Split**

In [16]:
X_train = new_train_data.iloc[:, 1:-1]
Y_train = new_train_data.iloc[:, -1]

X_test = test_data.iloc[:, 1:-1]

### **Original AdaBoost Model**
Trying default AdaBoost model, which performs well already with default parameters.
1. Applied **k-fold cross-validation** to build a more robust model.
2. Applied **GridSearchCV** to evaluate the optimal parameter
  - n_estimator: the number of weak classifier the adaboost model consist
  - learning_rate: how much the model react on wrong predicted samples
  - estimator: the weak classifier, usually uses decision tree
3. To understand the real-time process of grid search, I uses **iteration to print out the process of different parameter combination**

In [17]:
# In[] Original AdaBoost
model = AdaBoostClassifier()
#cv
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Accuracy: 0.809 (0.007)


In [39]:
# In[] Grid search with detailed reporting
model = AdaBoostClassifier()

# Define the grid of values to search
grid = dict()
grid['n_estimators'] = [30, 50, 100, 150, 200] # start higher due to class imbalance, some might not be included in one training
grid['learning_rate'] = [0.01, 0.05, 0.1, 0.3, 0.5]
grid['estimator'] = [DecisionTreeClassifier(max_depth=1),
                    DecisionTreeClassifier(max_depth=3),
                    DecisionTreeClassifier(max_depth=5),
                    DecisionTreeClassifier(max_depth=7),
                    DecisionTreeClassifier(max_depth=10),]

# Define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

# Define the grid search procedure with multiple scoring metrics
# grid_search = GridSearchCV(estimator=model,
#                          param_grid=grid,
#                          n_jobs=-1,
#                          cv=cv,
#                          scoring=['accuracy', 'recall_macro'],  # recall_macro for multi-classes
#                          refit='accuracy',  # Refit using accuracy
#                          return_train_score=True,
#                          verbose=3)

In [40]:
# Custom scorer: Recall for churn classes only (1-4)
def churn_recall(y_true, y_pred):
    mask = y_true > 0  # Churn samples
    if np.sum(mask) == 0:  # Edge case: no churn in fold
        return np.nan
    y_true_churn = y_true[mask]
    y_pred_churn = y_pred[mask]
    return recall_score(y_true_churn, y_pred_churn, average='macro', zero_division=0)

# Custom scorer: F1 for churn classes only (1-4)
def churn_f1(y_true, y_pred):
    mask = y_true > 0  # Churn samples
    if np.sum(mask) == 0:  # Edge case: no churn in fold
        return np.nan
    y_true_churn = y_true[mask]
    y_pred_churn = y_pred[mask]
    return f1_score(y_true_churn, y_pred_churn, average='macro', zero_division=0)

churn_recall_scorer = make_scorer(churn_recall)
churn_f1_scorer = make_scorer(churn_f1)

In [41]:
# Calculate total combinations
param_combinations = list(itertools.product(
    grid['n_estimators'],
    grid['learning_rate'],
    grid['estimator']
))
n_combinations = len(param_combinations)
print(f"Starting Grid Search with {n_combinations} combinations...")

# Store results
results = {
    'params': [],
    'mean_accuracy': [],
    'mean_recall_macro': [],
    'mean_churn_recall': [],
    'mean_f1_macro': [],
    'mean_churn_f1': []
}

# Progress bar with real-time updates
with tqdm(total=n_combinations, desc="Parameter combinations", unit="combo") as pbar:
    for idx, (n_est, lr, est) in enumerate(param_combinations, 1):
        current_model = AdaBoostClassifier(
            n_estimators=n_est,
            learning_rate=lr,
            estimator=est
        )

        # Cross-validate with multiple metrics
        scores = cross_validate(
            current_model,
            X_train,
            Y_train,
            cv=cv,
            scoring={
                'accuracy': 'accuracy',
                'recall_macro': 'recall_macro',
                'churn_recall': churn_recall_scorer,
                'f1_macro': 'f1_macro',
                'churn_f1': churn_f1_scorer
            },
            n_jobs=-1
        )

        # Extract mean scores
        mean_acc = np.mean(scores['test_accuracy'])
        mean_rec_macro = np.mean(scores['test_recall_macro'])
        mean_churn_rec = np.nanmean(scores['test_churn_recall'])
        mean_f1_macro = np.mean(scores['test_f1_macro'])
        mean_churn_f1 = np.nanmean(scores['test_churn_f1'])

        # Store results
        results['params'].append({'n_estimators': n_est, 'learning_rate': lr, 'estimator': est})
        results['mean_accuracy'].append(mean_acc)
        results['mean_recall_macro'].append(mean_rec_macro)
        results['mean_churn_recall'].append(mean_churn_rec)
        results['mean_f1_macro'].append(mean_f1_macro)
        results['mean_churn_f1'].append(mean_churn_f1)

        # Print progress
        print(f"\nCombination {idx}/{n_combinations}:")
        print(f"Parameters: n_estimators={n_est}, learning_rate={lr}, max_depth={est.max_depth}")
        print(f"Mean Accuracy: {mean_acc:.4f}")
        print(f"Mean Recall (macro, all classes): {mean_rec_macro:.4f}")
        print(f"Mean Churn Recall (classes 1-4): {mean_churn_rec:.4f}")
        print(f"Mean F1 (macro, all classes): {mean_f1_macro:.4f}")
        print(f"Mean Churn F1 (classes 1-4): {mean_churn_f1:.4f}")
        print("-" * 50)

        pbar.update(1)

# Find best combination based on churn F1 (or churn recall, your choice)
best_idx = np.argmax(results['mean_churn_f1'])  # Switch to 'mean_churn_recall' if preferred
best_params = results['params'][best_idx]
best_churn_f1 = results['mean_churn_f1'][best_idx]
best_churn_recall = results['mean_churn_recall'][best_idx]

print("\nBest Configuration (by Churn F1):")
print(f"Best Churn F1: {best_churn_f1:.4f}")
print(f"Best Churn Recall: {best_churn_recall:.4f}")
print(f"Best Parameters: n_estimators={best_params['n_estimators']}, "
      f"learning_rate={best_params['learning_rate']}, "
      f"max_depth={best_params['estimator'].max_depth}")

Starting Grid Search with 125 combinations...


Parameter combinations:   1%|          | 1/125 [00:06<12:42,  6.15s/combo]


Combination 1/125:
Parameters: n_estimators=30, learning_rate=0.01, max_depth=1
Mean Accuracy: 0.8086
Mean Recall (macro, all classes): 0.2684
Mean Churn Recall (classes 1-4): 0.1017
Mean F1 (macro, all classes): 0.2434
Mean Churn F1 (classes 1-4): 0.0884
--------------------------------------------------


Parameter combinations:   2%|▏         | 2/125 [00:16<17:14,  8.41s/combo]


Combination 2/125:
Parameters: n_estimators=30, learning_rate=0.01, max_depth=3
Mean Accuracy: 0.8076
Mean Recall (macro, all classes): 0.2705
Mean Churn Recall (classes 1-4): 0.1038
Mean F1 (macro, all classes): 0.2495
Mean Churn F1 (classes 1-4): 0.0946
--------------------------------------------------


...


Combination 123/125:
Parameters: n_estimators=200, learning_rate=0.5, max_depth=5
Mean Accuracy: 0.8018
Mean Recall (macro, all classes): 0.3049
Mean Churn Recall (classes 1-4): 0.1421
Mean F1 (macro, all classes): 0.3011
Mean Churn F1 (classes 1-4): 0.1499
--------------------------------------------------


Parameter combinations:  99%|█████████▉| 124/125 [1:32:35<01:36, 96.03s/combo]


Combination 124/125:
Parameters: n_estimators=200, learning_rate=0.5, max_depth=7
Mean Accuracy: 0.8076
Mean Recall (macro, all classes): 0.2982
Mean Churn Recall (classes 1-4): 0.1336
Mean F1 (macro, all classes): 0.2937
Mean Churn F1 (classes 1-4): 0.1402
--------------------------------------------------


Parameter combinations: 100%|██████████| 125/125 [1:35:21<00:00, 45.78s/combo] 


Combination 125/125:
Parameters: n_estimators=200, learning_rate=0.5, max_depth=10
Mean Accuracy: 0.8086
Mean Recall (macro, all classes): 0.2900
Mean Churn Recall (classes 1-4): 0.1245
Mean F1 (macro, all classes): 0.2843
Mean Churn F1 (classes 1-4): 0.1296
--------------------------------------------------

Best Configuration (by Churn F1):
Best Churn F1: 0.1606
Best Churn Recall: 0.1489
Best Parameters: n_estimators=30, learning_rate=0.5, max_depth=7


no recall score, due to probably class imbalance and cross-validation might make each fold conatiaining few or no sample of one class, causing recall error >> cross-validation lesser splits

## **SOMTE-EEN & SMOTE-ENC Technique**
To handel the i**mbalanced dataset** of 73% no churn and 27% of churn, I tried 2 different over-sampling/under-sampling techniques to create a more balanced dataset to identify churn more robustly.

SMOTE

The difference of 2 methods:
- **SMOTE-ENN**: combines ``SMOTE`` which scales up the minority samples, and ENN method that scale down the majority samples to create balanced dataset. For all data points, ``ENN``(Edited Nearest Neighbor) finds its nearest K neighbors,and clean the samples which have more then $\alpha$ (a threshold to be set) percent which are minority class, these samples get cleaned.
- **SMOTE-ENC**: for datasets with both nominal and continuous features, ``SMOTE-NC`` is the only SMOTE-based over-sampling technique to balance the data, and ``SMOTE-ENC ``method offers better prediction than model using ``SMOTE-NC`` when the dataset has a substantial number of nominal features and also when there is some association between the categorical features and the target class

In [ ]:
# In[] smoteen
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN()
X_resampled, Y_resampled = smote_enn.fit_resample(X_train, Y_train)
print(sorted(Counter(Y_resampled).items()))

In [ ]:
# In[] smotenc
from imblearn.over_sampling import SMOTENC, ADASYN
from collections import Counter
# from imblearn.combine import SMOTENC
smote_enc = SMOTENC(categorical_features=[0, 2, 3, 4, 5, 9, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])
X_resampled, Y_resampled = smote_enc.fit_resample(X_train, Y_train)
print(sorted(Counter(Y_resampled).items()))

In [ ]:
# In[]
model = AdaBoostClassifier(n_estimators=30, learning_rate=0.3, base_estimator=DecisionTreeClassifier(max_depth = 7))
model.fit(X_resampled, Y_resampled)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
n_scores = cross_val_score(model, X_resampled, Y_resampled, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

In [ ]:
# report performance
print('Accuracy(resampled): %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy(train): %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
# In[]
for lr in grid['learning_rate']:
    for e in range(1,6):
        model = AdaBoostClassifier(n_estimators = 300, learning_rate=lr, base_estimator=DecisionTreeClassifier(max_depth=e))
        model.fit(X_resampled, Y_resampled)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
        n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        print(lr, e)
        print('Accuracy(train): %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
# In[]
# from imblearn.ensemble import BalancedBaggingClassifier
# from imblearn.ensemble import EasyEnsembleClassifier
# # bbc = BalancedBaggingClassifier(n_estimators = 100)
# eec = EasyEnsembleClassifier(n_estimators = 100)
# model = eec.fit(X_train, Y_train)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Accuracy(train): %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
# In[] Final Predict
model = AdaBoostClassifier(n_estimators=100, learning_rate=0.3, base_estimator=DecisionTreeClassifier(max_depth = 1))
model.fit(X_resampled, Y_resampled)
Y_pred = model.predict(X_test)

In [ ]:
#f1-score
from sklearn.metrics import f1_score
f1_score = f1_score(Y_train, model.predict(X_train), average=None)
print(model.get_params())
print("f1-score:" , f1_score)

In [ ]:
#cross-validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
# In[]
output = np.concatenate((test_data.iloc[:, 0].to_numpy().reshape((1409,1)), Y_pred.reshape((1409,1))), axis=1)
output = pd.DataFrame(output)
output.iloc[:, 1] = output.iloc[:, 1].astype(np.int)
output = output.set_axis(['Customer ID','Churn Category'], axis=1)
output.to_csv('/Users/nina/Desktop/ML110-1/Adatest_smotenc_.csv', index=False)